In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)
%matplotlib inline

### Nivell 1

- Exercici 1

Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

In [3]:
url = 'https://www.bolsamadrid.es'
html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.content, 'html.parser')

In [5]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
meta
meta
meta
title
link
link
link
link
link
script
link
script
body
div
table
tr
td
div
ul
li
a
div
ul
li
a
li
a
li
a
li
a
img
tr
td
div
td
td
a
img
td
div
a
img
div
div
table
tr
td
div
a
div
div
br
form
input
div
span
a
span
div
td
td
div
tr
td
div
div
a
tr
td
script
div
div
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
a
div
div
div
a
div
div
a
div
div
div
a

+ anem a buscar els valors de les accions de les diferents companyies del IBEX35
+ posarem aquests valors en un dataFrame

In [6]:
# en primer lloc vaig a trobar tots els hyperlinks que tingin el string 'Acciones' 
links = []
for link in soup.find_all('a', string='Acciones'):
    links.append(link.get('href'))

In [7]:
links

['/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000',
 '/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000']

In [8]:
new_url = url+links[0]
new_url

'https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000'

In [9]:
html_acciones = requests.get(new_url)
soup_acciones = BeautifulSoup(html_acciones.content, 'html.parser')

In [10]:
# busco la taula de continguts amb el seguent id especific
tabla_acciones = soup_acciones.find(id='ctl00_Contenido_tblAcciones')

In [11]:
tabla_acciones.find_all('th')

[<th scope="col">Nombre</th>,
 <th scope="col">Últ.</th>,
 <th scope="col">% Dif.</th>,
 <th scope="col">Máx.</th>,
 <th scope="col">Mín.</th>,
 <th scope="col">Volumen</th>,
 <th scope="col">Efectivo (miles €)</th>,
 <th scope="col">Fecha</th>,
 <th class="Ult" scope="col">Hora</th>]

In [12]:
columnas=[]

for element in tabla_acciones.find_all("th"):    
    columnas.append(element.get_text())
    
print(columnas)

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [13]:
# guardo tots els valors en aquestas llista
resultados = []
for element in tabla_acciones.find_all("td"):
    resultados.append(element.text)    

In [14]:
# creo un diccionari
datos = {}

In [15]:
# em salto cada 9 elements per tenir els mateix tipus de valors
for i in range(9):
    datos[columnas[i]]=resultados[i::9]

In [16]:
#converteixo el diccionari en un DataFrame
df = pd.DataFrame(datos)

In [17]:
# ja tinc la taula de les 35 companyias del IBEX
df

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"135,7000","3,35","136,2000","131,7000",116.363,"15.656,34",14/06/2021,Cierre
1,ACERINOX,"11,6700","-0,26","11,8500","11,6700",567.980,"6.658,85",14/06/2021,Cierre
2,ACS,"25,5700","0,59","25,8300","25,5000",1.445.997,"36.991,83",14/06/2021,Cierre
3,AENA,"143,1000","1,89","143,1000","140,5000",118.328,"16.891,45",14/06/2021,Cierre
4,ALMIRALL,"14,8700","-0,60","15,0600","14,8200",196.648,"2.927,87",14/06/2021,Cierre
5,AMADEUS,"65,5000","-0,30","66,3800","65,2600",855.817,"56.118,56",14/06/2021,Cierre
6,ARCELORMIT.,"26,9350","-1,23","27,6300","26,9350",316.010,"8.593,06",14/06/2021,Cierre
7,B.SANTANDER,"3,4375","0,09","3,4675","3,4160",38.788.194,"133.361,07",14/06/2021,Cierre
8,BA.SABADELL,"0,6404","0,06","0,6470","0,6360",17.116.266,"10.978,94",14/06/2021,Cierre
9,BANKINTER,"4,5100","0,94","4,5240","4,4790",1.917.939,"8.646,52",14/06/2021,Cierre


In [18]:
df.to_csv('Acciones_hoy_ibex.csv')
# save data to csv

+ anem a fer una cosa similar amb selenium

In [19]:
# obrim la pagina web a la url disitjada

from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.set_headless()
assert opts.headless  # Operating in headless mode
#browser = Firefox(options=opts)
browser = Firefox(executable_path = '/home/vant/Descargas/geckodriver-v0.29.1-linux64/geckodriver')
url = 'https://www.bolsamadrid.es'
browser.get(url)

<ipython-input-19-a91e243008e7>:6: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [20]:
# busquem el link on tenim el string "Acciones" i cliquem
links = browser.find_elements_by_link_text('Acciones')
links[0].click()

In [21]:
# un cop a la nova finestra busquem els valors de la taula
table = browser.find_element_by_id('ctl00_Contenido_tblAcciones')

In [22]:
columnas=[]

for element in table.find_elements_by_tag_name("th"):    
    columnas.append(element.text)
    
print(columnas)    

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [23]:
# guardo tots els valors en aquestas llista
resultados = []
for element in table.find_elements_by_tag_name("td"):
    resultados.append(element.text)    

In [24]:
# em salto cada 9 elements per tenir els mateix tipus de valors
datos = {}
for i in range(9):
    datos[columnas[i]]=resultados[i::9]

pd.DataFrame(datos)    

,Nombre,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
0,ACCIONA,"135,7000","3,35","136,2000","131,7000",116.363,"15.656,34",14/06/2021,Cierre
1,ACERINOX,"11,6700","-0,26","11,8500","11,6700",567.980,"6.658,85",14/06/2021,Cierre
2,ACS,"25,5700","0,59","25,8300","25,5000",1.445.997,"36.991,83",14/06/2021,Cierre
3,AENA,"143,1000","1,89","143,1000","140,5000",118.328,"16.891,45",14/06/2021,Cierre
4,ALMIRALL,"14,8700","-0,60","15,0600","14,8200",196.648,"2.927,87",14/06/2021,Cierre
5,AMADEUS,"65,5000","-0,30","66,3800","65,2600",855.817,"56.118,56",14/06/2021,Cierre
6,ARCELORMIT.,"26,9350","-1,23","27,6300","26,9350",316.010,"8.593,06",14/06/2021,Cierre
7,B.SANTANDER,"3,4375","0,09","3,4675","3,4160",38.788.194,"133.361,07",14/06/2021,Cierre
8,BA.SABADELL,"0,6404","0,06","0,6470","0,6360",17.116.266,"10.978,94",14/06/2021,Cierre
9,BANKINTER,"4,5100","0,94","4,5240","4,4790",1.917.939,"8.646,52",14/06/2021,Cierre


In [25]:
df.to_csv('Acciones_hoy_ibex_v2.csv')
# save data to csv

In [26]:
browser.close()

### Nivell 2

- Exercici 2

Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

In [27]:
#la informacio de text es pot trobar al word adjunt

In [127]:
df =pd.read_csv('Acciones_hoy_ibex_v2.csv',index_col='Nombre')
df.drop('Unnamed: 0',axis=1,inplace=True)

In [128]:
df.head()

,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
Nombre,,,,,,,,
ACCIONA,"135,7000","3,35","136,2000","131,7000",116.363,"15.656,34",14/06/2021,Cierre
ACERINOX,"11,6700","-0,26","11,8500","11,6700",567.980,"6.658,85",14/06/2021,Cierre
ACS,"25,5700","0,59","25,8300","25,5000",1.445.997,"36.991,83",14/06/2021,Cierre
AENA,"143,1000","1,89","143,1000","140,5000",118.328,"16.891,45",14/06/2021,Cierre
ALMIRALL,"14,8700","-0,60","15,0600","14,8200",196.648,"2.927,87",14/06/2021,Cierre


In [129]:
df['Últ.'] =df['Últ.'].str.replace(',','.').astype(float)
df['% Dif.'] =df['% Dif.'].str.replace(',','.').astype(float)
df['Máx.'] =df['Máx.'].str.replace(',','.').astype(float)
df['Mín.'] =df['Mín.'].str.replace(',','.').astype(float)
df['Efectivo (miles €)'] =df['Efectivo (miles €)'].str.replace('.','').str.replace(',','.').astype(float)

In [130]:
df.head()

,Últ.,% Dif.,Máx.,Mín.,Volumen,Efectivo (miles €),Fecha,Hora
Nombre,,,,,,,,
ACCIONA,135.70,3.35,136.20,131.70,116.363,15656.34,14/06/2021,Cierre
ACERINOX,11.67,-0.26,11.85,11.67,567.980,6658.85,14/06/2021,Cierre
ACS,25.57,0.59,25.83,25.50,1.445.997,36991.83,14/06/2021,Cierre
AENA,143.10,1.89,143.10,140.50,118.328,16891.45,14/06/2021,Cierre
ALMIRALL,14.87,-0.60,15.06,14.82,196.648,2927.87,14/06/2021,Cierre


In [133]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, ACCIONA to VISCOFAN
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Últ.                35 non-null     float64
 1   % Dif.              35 non-null     float64
 2   Máx.                35 non-null     float64
 3   Mín.                35 non-null     float64
 4   Volumen             35 non-null     object 
 5   Efectivo (miles €)  35 non-null     float64
 6   Fecha               35 non-null     object 
 7   Hora                35 non-null     object 
dtypes: float64(5), object(3)
memory usage: 2.5+ KB


### Nivell 3


- Exercici 3

Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 

In [29]:
import scrapy

In [30]:
url ='https://www.elperiodico.com/es/'